# Python Assignment: K-Means Clustering and Elbow Method Implementation

This assignment will challenge your understanding of unsupervised learning by requiring you to implement the K-Means clustering algorithm and the Elbow Method from scratch (or primarily using `numpy` for numerical operations). You will generate synthetic datasets, apply your implementation, and visualize the results. This will solidify your grasp of the algorithm's mechanics and the heuristics for determining optimal cluster counts.

## Part 1: Data Generation (15 points)

We'll start by generating synthetic 2D datasets that will be used for clustering.

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from sklearn.datasets import make_blobs
from sklearn.preprocessing import StandardScaler

np.random.seed(42) # for reproducibility

# 1.1 Generate Dataset 1: Distinct Blobs
#    Generate a dataset `X1` with 3 well-separated clusters.
#    - `n_samples`: 500
#    - `n_features`: 2
#    - `centers`: 3
#    - `cluster_std`: 0.8
#    Visualize `X1` using a scatter plot. Ensure axes are labeled.

X1, y1 = make_blobs(
    # Your parameters here
)

plt.figure(figsize=(8, 6))
# Your visualization code here
plt.title("Dataset 1: Distinct Blobs")
plt.xlabel("Feature 1")
plt.ylabel("Feature 2")
plt.grid(True)
plt.show()


# 1.2 Generate Dataset 2: Overlapping Blobs
#    Generate a dataset `X2` with 4 clusters that are somewhat overlapping.
#    - `n_samples`: 700
#    - `n_features`: 2
#    - `centers`: 4
#    - `cluster_std`: 1.5 (increase this to create overlap)
#    Visualize `X2` using a scatter plot.

X2, y2 = make_blobs(
    # Your parameters here
)

plt.figure(figsize=(8, 6))
# Your visualization code here
plt.title("Dataset 2: Overlapping Blobs")
plt.xlabel("Feature 1")
plt.ylabel("Feature 2")
plt.grid(True)
plt.show()


# 1.3 Preprocessing: Standardization
#    Standardize both datasets `X1` and `X2` using `StandardScaler` from `sklearn.preprocessing`.
#    Store the standardized data in `X1_scaled` and `X2_scaled`.
#    Explain why standardization is important for K-Means clustering.

# Your standardization code here

print("X1_scaled shape:", X1_scaled.shape)
print("X2_scaled shape:", X2_scaled.shape)

### Explanation: Why Standardization for K-Means?
*(Write your explanation here)*


## Part 2: K-Means Clustering Implementation (40 points)

Implement the K-Means algorithm from scratch. You are allowed to use `numpy` for numerical operations (e.g., array manipulations, distance calculations, mean). Avoid using `sklearn.cluster.KMeans` for the core algorithm.

In [ ]:
class KMeans:
    def __init__(self, n_clusters: int, max_iter: int = 300, tol: float = 1e-4):
        self.n_clusters = n_clusters
        self.max_iter = max_iter
        self.tol = tol
        self.centroids = None
        self.labels = None
        self.inertia_ = None # Sum of squared distances of samples to their closest cluster center

    def _initialize_centroids(self, X: np.ndarray) -> np.ndarray:
        # TODO: Implement K-Means++ initialization.
        # K-Means++ is a smarter way to initialize centroids to speed up convergence
        # and avoid poor local optima.
        # Steps:
        # 1. Choose one center uniformly at random from the data points.
        # 2. For each data point x, compute D(x), the distance between x and the nearest center that has already been chosen.
        # 3. Choose one new data point as a new center, with probability proportional to D(x)^2.
        # 4. Repeat steps 2 and 3 until k centers have been chosen.

        # If K-Means++ is too challenging, fall back to random initialization,
        # but clearly state that it's random initialization.

        # Random initialization (fallback):
        # indices = np.random.choice(X.shape[0], self.n_clusters, replace=False)
        # return X[indices]

        # K-Means++ implementation (highly recommended):
        n_samples, n_features = X.shape
        centroids = np.zeros((self.n_clusters, n_features))

        # 1. Choose first centroid uniformly at random
        first_idx = np.random.randint(0, n_samples)
        centroids[0] = X[first_idx]

        # 2. Iteratively choose other centroids
        for i in range(1, self.n_clusters):
            # Compute distances from each point to the nearest existing centroid
            distances = np.min(self._euclidean_distance(X, centroids[:i]), axis=1)
            # Probabilities proportional to D(x)^2
            probabilities = distances**2 / np.sum(distances**2)
            # Choose new centroid based on probabilities
            new_centroid_idx = np.random.choice(n_samples, p=probabilities)
            centroids[i] = X[new_centroid_idx]
        return centroids


    def _euclidean_distance(self, X: np.ndarray, centroids: np.ndarray) -> np.ndarray:
        # TODO: Calculate Euclidean distance between each data point in X and each centroid.
        # X: (n_samples, n_features)
        # centroids: (n_clusters, n_features)
        # Returns: (n_samples, n_clusters) matrix of distances

        # Hint: Use broadcasting or np.linalg.norm
        # Example: np.linalg.norm(X[:, np.newaxis] - centroids, axis=2)
        return np.linalg.norm(X[:, np.newaxis] - centroids, axis=2)

    def _assign_clusters(self, X: np.ndarray, centroids: np.ndarray) -> np.ndarray:
        # TODO: Assign each data point to the closest centroid.
        # Returns: (n_samples,) array of cluster labels (integers from 0 to n_clusters-1)

        distances = self._euclidean_distance(X, centroids)
        return np.argmin(distances, axis=1)

    def _update_centroids(self, X: np.ndarray, labels: np.ndarray) -> np.ndarray:
        # TODO: Calculate new centroids as the mean of all points assigned to each cluster.
        # Handle empty clusters (e.g., re-initialize them randomly or keep their old position).
        # For this assignment, if a cluster becomes empty, you can either:
        # 1. Keep its old centroid position.
        # 2. Re-initialize it randomly from the data points.
        # Option 1 is simpler for initial implementation.

        new_centroids = np.zeros_like(self.centroids)
        for i in range(self.n_clusters):
            points_in_cluster = X[labels == i]
            if len(points_in_cluster) > 0:
                new_centroids[i] = np.mean(points_in_cluster, axis=0)
            else:
                # Handle empty cluster: Keep old centroid for now.
                new_centroids[i] = self.centroids[i]
        return new_centroids

    def fit(self, X: np.ndarray):
        if X.ndim != 2:
            raise ValueError("Input data X must be 2-dimensional (n_samples, n_features).")

        self.centroids = self._initialize_centroids(X)

        for i in range(self.max_iter):
            old_centroids = self.centroids.copy()

            self.labels = self._assign_clusters(X, self.centroids)
            self.centroids = self._update_centroids(X, self.labels)

            # Check for convergence
            if np.linalg.norm(self.centroids - old_centroids) < self.tol:
                print(f"K-Means converged after {i+1} iterations.")
                break

        # Calculate final inertia
        distances_to_centroids = self._euclidean_distance(X, self.centroids)
        self.inertia_ = np.sum(distances_to_centroids[np.arange(len(X)), self.labels]**2)

    def predict(self, X: np.ndarray) -> np.ndarray:
        if self.centroids is None:
            raise ValueError("Model not fitted yet. Call fit() first.")
        return self._assign_clusters(X, self.centroids)


# Test your KMeans implementation with Dataset 1 (X1_scaled)
print("\n--- Testing K-Means on X1_scaled ---")
kmeans1 = KMeans(n_clusters=3, max_iter=300, tol=1e-4)
kmeans1.fit(X1_scaled)

print("Final Centroids for X1_scaled:\n", kmeans1.centroids)
print("Final Inertia for X1_scaled:", kmeans1.inertia_)

# Visualize the results for X1_scaled
plt.figure(figsize=(8, 6))
plt.scatter(X1_scaled[:, 0], X1_scaled[:, 1], c=kmeans1.labels, cmap='viridis', s=50, alpha=0.8)
plt.scatter(kmeans1.centroids[:, 0], kmeans1.centroids[:, 1], marker='X', s=200, color='red', label='Centroids')
plt.title("K-Means Clustering on X1_scaled (K=3)")
plt.xlabel("Feature 1 (Scaled)")
plt.ylabel("Feature 2 (Scaled)")
plt.legend()
plt.grid(True)
plt.show()


# Test your KMeans implementation with Dataset 2 (X2_scaled)
print("\n--- Testing K-Means on X2_scaled ---")
kmeans2 = KMeans(n_clusters=4, max_iter=300, tol=1e-4)
kmeans2.fit(X2_scaled)

print("Final Centroids for X2_scaled:\n", kmeans2.centroids)
print("Final Inertia for X2_scaled:", kmeans2.inertia_)

# Visualize the results for X2_scaled
plt.figure(figsize=(8, 6))
plt.scatter(X2_scaled[:, 0], X2_scaled[:, 1], c=kmeans2.labels, cmap='viridis', s=50, alpha=0.8)
plt.scatter(kmeans2.centroids[:, 0], kmeans2.centroids[:, 1], marker='X', s=200, color='red', label='Centroids')
plt.title("K-Means Clustering on X2_scaled (K=4)")
plt.xlabel("Feature 1 (Scaled)")
plt.ylabel("Feature 2 (Scaled)")
plt.legend()
plt.grid(True)
plt.show()

## Part 3: Elbow Method Implementation (35 points)

Implement the Elbow Method to find the optimal number of clusters for a dataset. This method relies on the within-cluster sum of squares (WCSS), which is often referred to as 'inertia' in `sklearn`.

In [ ]:
def run_elbow_method(X: np.ndarray, max_k: int) -> list:
    # TODO: Implement the Elbow Method.
    # Iterate K from 1 to `max_k`.
    # For each K, run your `KMeans` implementation.
    # Store the `inertia_` (WCSS) value.
    # Return a list of inertia values for each K.

    inertias = []
    for k in range(1, max_k + 1):
        print(f"Running K-Means for K={k}...")
        kmeans = KMeans(n_clusters=k, max_iter=300, tol=1e-4)
        kmeans.fit(X)
        inertias.append(kmeans.inertia_)
    return inertias


# 3.1 Apply Elbow Method to Dataset 1 (X1_scaled)
#    Run the elbow method for `max_k` up to 10.
#    Plot the inertia values against K. Clearly label the plot.
#    Based on the plot, suggest the optimal number of clusters for X1_scaled.

max_k1 = 10
inertias1 = run_elbow_method(X1_scaled, max_k1)

plt.figure(figsize=(10, 6))
plt.plot(range(1, max_k1 + 1), inertias1, marker='o', linestyle='-')
plt.title("Elbow Method for X1_scaled")
plt.xlabel("Number of Clusters (K)")
plt.ylabel("Inertia (WCSS)")
plt.xticks(range(1, max_k1 + 1))
plt.grid(True)
plt.show()

### Optimal K for X1_scaled:
*(State your optimal K here and justify your choice based on the plot.)*


# 3.2 Apply Elbow Method to Dataset 2 (X2_scaled)
#    Run the elbow method for `max_k` up to 10.
#    Plot the inertia values against K.
#    Based on the plot, suggest the optimal number of clusters for X2_scaled.
#    Discuss the challenges of interpreting the elbow curve for overlapping clusters.

max_k2 = 10
inertias2 = run_elbow_method(X2_scaled, max_k2)

plt.figure(figsize=(10, 6))
plt.plot(range(1, max_k2 + 1), inertias2, marker='o', linestyle='-')
plt.title("Elbow Method for X2_scaled")
plt.xlabel("Number of Clusters (K)")
plt.ylabel("Inertia (WCSS)")
plt.xticks(range(1, max_k2 + 1))
plt.grid(True)
plt.show()

### Optimal K for X2_scaled:
*(State your optimal K here and justify your choice based on the plot.)*

### Discussion: Challenges of Elbow Method for Overlapping Clusters:
*(Write your discussion here)*

## Part 4: Advanced Challenges & Reflection (10 points)

Consider the limitations and potential improvements of your K-Means implementation and the Elbow Method.

In [ ]:
# 4.1 Multiple Runs (Conceptual)
#    K-Means is sensitive to initial centroid placement. How would you modify your `KMeans` class
#    or the `fit` method to run the algorithm multiple times with different initializations
#    and select the best result (e.g., the one with the lowest inertia)?

### Your Answer:
*(Write your conceptual answer here)*


# 4.2 Limitations of K-Means
#    List at least three inherent limitations of the K-Means clustering algorithm itself.

### Your Answer:
1.  *(Limitation 1)*
2.  *(Limitation 2)*
3.  *(Limitation 3)*


# 4.3 Alternative Evaluation Metrics (Conceptual)
#    Besides the Elbow Method's inertia, what other metrics or techniques can be used
#    to evaluate clustering performance, especially when true labels are not available?
#    Name at least two and briefly explain their purpose.

### Your Answer:
1.  *(Metric 1 and purpose)*
2.  *(Metric 2 and purpose)*
